## Tokenize

由于是新的文章所以需要先训练一个tokenizer

### Preprocess

In [1]:
with open("./contents/oscar.eo.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    
#preprocessing for DatasetforNSP, this is not the original dataset I would be training on. I am running this model as POC which I can adapt to any dataset later
#nothing useful or new here just a preprocessing step to make it same as same format I would be using
lines3=[] 
space = 0
tot =0
for i in lines:
    tot = tot+1
    if i.strip(" ")=="\n":
        continue
    if tot==4:
        lines3.append("\n")
        tot=0
        continue
    lines3.append(i)

In [2]:
with open("oscar.eo.txt", "w", encoding="utf-8")as f:
    for i in lines3:
        f.write(i)

In [3]:
from pathlib import Path
paths = ["oscar.eo.txt"]

### Training a tokenizers

In [4]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    strip_accents=True,
    lowercase=False,
)

# And then train
tokenizer.train(
    paths,
    vocab_size=50000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=1000,
    wordpieces_prefix="##",
)

In [5]:
!mkdir Bert
tokenizer.save_model("Bert", "voc")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
mkdir: Bert: File exists


['Bert/voc-vocab.txt']

In [6]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast(
    vocab_file= "./contents/Bert/voc-vocab.txt",
    do_lower_case = False,
    max_len=512
)

idx = tokenizer.encode("Mi estas Julien.")
tokenizer.convert_ids_to_tokens(idx)

['[CLS]', 'Mi', 'estas', 'Julie', '##n', '.', '[SEP]']

## Bert Model

In [7]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

False

In [8]:
from transformers import BertConfig

config = BertConfig(
    vocab_size=50000,
    hidden_size=768,
    num_hidden_layers=12, 
    num_attention_heads=12, 
    intermediate_size=3072, 
    hidden_act='gelu', 
    hidden_dropout_prob=0.1, 
    attention_probs_dropout_prob=0.1, 
    max_position_embeddings=512, 
    type_vocab_size=1, 
    initializer_range=0.02, 
    layer_norm_eps=1e-12, 
    pad_token_id=0, 
    gradient_checkpointing=False,
)

In [15]:
from model.mlm import BaseMLM

model = BaseMLM(vocab_size=50000, pretrain=None)()
model.num_parameters()

124492880

### Prepare Dataset

In [16]:
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
dataset = LineByLineTextDataset(tokenizer=tokenizer, 
                                file_path="./oscar.eo.txt", 
                                block_size=512 # maximum sequence length
                               )
print('No. of lines: ', len(dataset)) # No of lines in your datset

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

/Users/maochenhui/miniforge3/envs/DL/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Creating features from dataset file at ./oscar.eo.txt


No. of lines:  938226


In [ ]:
from train.mlm import MLMTrainer

trainer = MLMTrainer(model=model, 
                     data_collator=data_collator, 
                     train_dataset=dataset,
                     eval_dataset=None
                    )
t = trainer()
t.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 938226
  Num Epochs = 100
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1466000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
